In [1]:
import os
import sys

# Add the parent directory to the Python path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

# Now you can import the module from the examples directory
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from jax import vmap
from functools import partial
from scipy.spatial import distance_matrix
from l2ws.utils.nn_utils import get_nearest_neighbors
from l2ws.l2ws_model import L2WSmodel
from l2ws.algo_steps import create_k_steps_train, create_k_steps_eval

/Users/rajivs/Documents/Princeton/l2ws_conic/env/lib/python3.9/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


## Projected gradient descent

In this tutorial we show how to use our package to learn high-quality warm starts for your favorite fixed-point algorithm.
We consider projected gradient descent to solve the problem

\begin{array}{ll} \rm{minimize} &f_\theta(z),
&\\ \quad z \in \mathcal{C}
 \end{array}
where $z \in \mathbf{R}^n$ is the decision variable, $f_\theta : \mathbf{R}^n \rightarrow \mathbf{R}$ is a convex function and $L$-smooth function.

The iterates of projected gradient descent are
\begin{equation}
    z^{i+1} = \Pi_{\mathcal{C}}(z^i - \alpha \nabla f_\theta(z^i)).
\end{equation}

We take the specific example of non-negative least squares

\begin{array}{ll} \rm{minimize} &\|Az-b\|_2^2,
&\\ \quad z \geq 0
 \end{array}
where $A \in \mathbf{R}^{m \times n}$ and $b \in \mathbf{R}^{m}$ are problem data.
Here, $\theta = b$ is the parameter.

In [68]:
# setup
N_train = 5000
N_test = 100
N = N_train + N_test
m, n = 50, 25

# generate a single random A matrix
A = jnp.array(np.random.normal(size=(m, n)))

# generate N different b vectors
b_mat = jnp.array(np.random.normal(size=(N, m)))

k = 500
z0 = jnp.zeros(n)
evals, evecs = jnp.linalg.eigh(A.T @ A)
step = 1 / jnp.max(evals)

# setup inputs
b_mat_train = b_mat[:N_train, :]
b_mat_test = b_mat[N_train:, :]
train_inputs = b_mat_train
test_inputs = b_mat_test

In [69]:
# code up the fixed-point algorithm
def proj_gd_fixed_point(z, theta):
    return jnp.clip(z - step * A.T @ (A @ z - theta), a_min=0)

k_steps_train_proj_gd = create_k_steps_train(proj_gd_fixed_point)
k_steps_eval_proj_gd = create_k_steps_eval(proj_gd_fixed_point)

In [70]:
# solve all of the problems to get z_stars_train and z_stars_test
batch_k_steps_train_proj_gd = vmap(k_steps_train_proj_gd, in_axes=(None, 0, 0, None, None, None), out_axes=(0, 0))

z0_init_mat = jnp.zeros((N, n))
z_finals, iter_losses = batch_k_steps_train_proj_gd(1000, z0_init_mat, b_mat, False, None, False)

z_stars_train = z_finals[:N_train, :]
z_stars_test = z_finals[N_train:, :]

In [71]:
# create a ProjGDmodel object that inherits from the L2WSmodel
class ProjGDmodel(L2WSmodel):
    def __init__(self, input_dict):
        # self.fista = input_dict['algorithm'] == 'fista'
        super(ProjGDmodel, self).__init__(input_dict)

    def initialize_algo(self, input_dict):
        self.factor_static = None
        self.algo = 'proj_gd'
        self.factors_required = False
        self.q_mat_train, self.q_mat_test = input_dict['b_mat_train'], input_dict['b_mat_test']
        self.output_size = n

        self.k_steps_train_fn = partial(k_steps_train_proj_gd, jit=self.jit)
        self.k_steps_eval_fn = partial(k_steps_eval_proj_gd, jit=self.jit)
        self.out_axes_length = 4


In [77]:
# create l2ws_model
nn_cfg = {'lr': 1e-3, 'method': 'adam', 'intermediate_layer_sizes': [100]}
train_unrolls = 15
supervised = False
input_dict = dict(algorithm='proj_gd',
                    supervised=supervised,
                    train_unrolls=train_unrolls, 
                    jit=True,
                    train_inputs=train_inputs, 
                    test_inputs=test_inputs,
                    b_mat_train=b_mat_train, 
                    b_mat_test=b_mat_test,
                    nn_cfg=nn_cfg,
                    z_stars_train=z_stars_train,
                    z_stars_test=z_stars_test,
                    )
proj_gd_model = ProjGDmodel(input_dict)

In [78]:
# cold start evaluation
k = 100
init_eval_out = proj_gd_model.evaluate(
    k, test_inputs, b_mat_test, z_stars=z_stars_test, fixed_ws=False, tag='test')
init_test_losses = init_eval_out[1][1].mean(axis=0)

In [79]:
# nearest neighbor
# train_inputs, test_inputs = theta_mat_train, theta_mat_test

# full evaluation on the test set with nearest neighbor

nearest_neighbors_z = get_nearest_neighbors(train_inputs, test_inputs, z_stars_train)
nn_eval_out = proj_gd_model.evaluate(k, nearest_neighbors_z,
                                    b_mat_test, z_stars=z_stars_test,
                                    fixed_ws=True, tag='test')
nn_losses = nn_eval_out[1][1].mean(axis=0)

distances [[ 9.93193168  9.62785156 10.80164922 ...  8.32819901  9.53913757
   9.49720965]
 [10.58304128 11.44594845 13.22317106 ... 10.62330622 12.38117561
  10.03758508]
 [11.47430542 12.75321912 10.27950395 ...  9.84002225  9.84028938
   9.76634099]
 ...
 [10.76189671 10.00784086 10.20048707 ...  9.05181563 10.26836854
   8.56355041]
 [10.206239   10.71995581 11.01504408 ...  9.36942741  9.69739709
  10.52457185]
 [10.15974531 10.70140221 12.30301967 ...  7.97272642 11.06015278
   9.45495608]]
indices [ 922  845  653 2952 2498 2289 4340 2280 2019 3901 4635 3045 4231 2953
  539  381  360 3344 3927 1520 4415 1563 3854 3332 3498 2692 1179 1556
 1126 1378 1207 1363 2474 4749 3121 2847  629 3515 3251 4452 4255 2134
 4433 4784 3231  728 4564 3212 1218 3324 1956 1390 1759 4521  160 3850
 3341 4061 2825 4415 1032  406 4870 4659 4808 1179  817 4932 4954 2871
  470  806 3635 4416 2956 4438 4934 2324 1422 4755 2166 1123 3948 3414
 2996  645 4403  595 4954  862 2825 3231  756 2564 3028  293 475

In [80]:
# train the osqp_model
# call train_batch without jitting
params, state = proj_gd_model.params, proj_gd_model.state
num_epochs = 1000
train_losses = jnp.zeros(num_epochs)
for i in range(num_epochs):
    train_result = proj_gd_model.train_full_batch(params, state)
    loss, params, state = train_result
    train_losses = train_losses.at[i].set(loss)

proj_gd_model.params, proj_gd_model.state = params, state

# full evaluation on the test set
final_eval_out = proj_gd_model.evaluate(
    k, test_inputs, b_mat_test, z_stars=z_stars_test, fixed_ws=False, tag='test')
final_test_losses = final_eval_out[1][1].mean(axis=0)

In [98]:
colors = plt.cm.Set1

plt.plot(final_test_losses, label=f"learned k={train_unrolls}")
plt.plot(nn_losses, label='nearest neighbor')
plt.plot(init_test_losses, label='cold start')
plt.yscale('log')
plt.title('fixed-point residuals')
plt.ylabel('test fixed-point residuals')
plt.xlabel('evaluation iterations')
plt.legend()
plt.show()

plt.plot(init_test_losses / final_test_losses, label=f"learned k={train_unrolls}")
plt.plot(init_test_losses / nn_losses, label='nearest neighbor')
plt.title('gains')
plt.ylabel('test gain over the cold start')
plt.xlabel('evaluation iterations')
plt.legend()
plt.show()

label=r'train loss: $\ell^{\rm reg}_\theta$ in this case' if supervised else r'train loss: $\ell^{\rm fp}_\theta$ in this case'
plt.plot(train_losses, label=label, color='magenta')
plt.yscale('log')
test_losses = np.array([init_test_losses[train_unrolls], final_test_losses[train_unrolls]])
epochs_array = np.array([0, num_epochs])

test_label = r'test fixed-point residual: $\ell^{\rm fp}_\theta$'
plt.plot(epochs_array, test_losses, label=test_label, color='brown')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.yscale('log')
plt.title('training and test losses')
plt.legend()
plt.show()

AttributeError: 'ColormapRegistry' object has no attribute 'set1'